In [1]:
import random
import operator
import csv
import itertools

import numpy

#---------------import for plotting------------------
import matplotlib.pyplot as plt
import networkx as nx
import pydot
from networkx.drawing.nx_pydot import graphviz_layout
#----------------------------------------------------

from deap import alteralgorithms as algorithms
from deap import base
from deap import creator
from deap import tools
from deap import altergp as gp

# Read the iris list features and put it in a list of lists.
with open("iris.csv") as irisbase:
    irisReader = csv.reader(irisbase)
    irisDB = list(list(str(elem) if elem == "Iris-setosa" or elem == "Iris-versicolor" or elem == "Iris-virginica" else float(elem) for elem in row) for row in irisReader)

#Matrix of variables which has Min and Max per col
varMatrix=[]
for n in range(len(irisDB[0])-1):
    o=[]

    for elem in irisDB:
        o.append(*elem[n:n+1])

    varMatrix.append([numpy.max(o), numpy.min(o)])

maxTotal = numpy.max(varMatrix)
minTotal = numpy.min(varMatrix)

# Iris matrix: Iris-setosa = True; Iris-versicolor AND Iris-virginica = False
iris = list(list(bool(True) if elem == "Iris-setosa" else bool(False) if elem == "Iris-versicolor" or elem == "Iris-virginica" else float(elem) for elem in row) for row in irisDB)

# Iris matrix without class Iris-setosa
irisNoSetosa = list(list(bool(True) if elem == "Iris-versicolor" else bool(False) if elem == "Iris-virginica" else float(elem) for elem in row) for row in irisDB if row[4] != "Iris-setosa")

# defined a new primitive set for strongly typed GP
pset = gp.PrimitiveSetTyped("MAIN", itertools.repeat(float, 4), bool)
pset.renameArguments(ARG0="SepalLength")
pset.renameArguments(ARG1="SepalWidth")
pset.renameArguments(ARG2="PetalLength")
pset.renameArguments(ARG3="PetalWidth")

# boolean operators
pset.addPrimitive(operator.and_, [bool, bool], bool, "And")
pset.addPrimitive(operator.or_, [bool, bool], bool, "Or")

# logic operators
class ERC(object): pass
pset.addPrimitive(operator.lt, [float, ERC], bool)
pset.addPrimitive(operator.gt, [float, ERC], bool)
pset.addPrimitive(operator.le, [float, ERC], bool)
pset.addPrimitive(operator.ge, [float, ERC], bool)
pset.addPrimitive(operator.eq, [float, ERC], bool)
pset.addPrimitive(operator.ne, [float, ERC], bool)

def In(input, leftmost, rightmost):
    if leftmost > rightmost:
        aux = leftmost
        leftmost = rightmost
        rightmost = aux
    
    if leftmost < input and input < rightmost:
        return True
    else:
        return False

def Out(input, leftmost, rightmost):
    if leftmost > rightmost:
        aux = leftmost
        leftmost = rightmost
        rightmost = aux
    
    if leftmost < input and input < rightmost:
        return False
    else:
        return True

pset.addPrimitive(In, [float, ERC, ERC], bool)
pset.addPrimitive(Out, [float, ERC, ERC], bool)


# terminals
pset.addEphemeralConstant("rand", lambda: random.uniform(minTotal, maxTotal), ERC)
pset.addTerminal(True, bool)
pset.addTerminal(False, bool)

#print (pset.primitives.keys())
#print (pset.terminals.keys())

# Define the operator fitnessMax and create the individual who uses it
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=6)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def evalIrisbase(individual, matrix):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    #print (str(individual))
    #print("")
    result = sum(bool(func(*elem[:4])) is bool(elem[4]) for elem in matrix)

    return result,
    
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=1, max_=6)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))


def plotting(individual):
	nodes, edges, labels = gp.graph(individual)
	g = nx.Graph()
	g.add_nodes_from(nodes)
	g.add_edges_from(edges)
	pos = graphviz_layout(g, prog="dot")

	nx.draw_networkx_nodes(g, pos)
	nx.draw_networkx_edges(g, pos)
	nx.draw_networkx_labels(g, pos, labels)
	plt.show()


def main():
    random.seed(40)

    #----------Iris-setosa vs. Iris-versicolor, Iris-virginica----------
    pop_1 = toolbox.population(n=500)
    hof_1 = tools.HallOfFame(1)

    stats_1 = tools.Statistics(lambda ind: ind.fitness.values)
    stats_1.register("avg", numpy.mean)
    stats_1.register("std", numpy.std)
    stats_1.register("min", numpy.min)
    stats_1.register("max", numpy.max)
    
    toolbox.register("evaluate", evalIrisbase, matrix=iris)

    print(str("Iris-setosa vs. Iris-versicolor, Iris-virginica:"))
    algorithms.eaSimple(pop_1, toolbox, 0.5, 0.2, 100, stats_1, halloffame=hof_1, verbose=True, reportFrec=10)
    
    print("")
    print("HallOfFame:")
    print (hof_1[0])
    print("")

    #plotting(hof_1[0])

    #----------Iris-versicolor vs. Iris-virginica----------
    pop_2 = toolbox.population(n=500)
    hof_2 = tools.HallOfFame(1)

    stats_2 = tools.Statistics(lambda ind: ind.fitness.values)
    stats_2.register("avg", numpy.mean)
    stats_2.register("std", numpy.std)
    stats_2.register("min", numpy.min)
    stats_2.register("max", numpy.max)
    
    toolbox.register("evaluate", evalIrisbase, matrix=irisNoSetosa)

    print(str("Iris-versicolor vs. Iris-virginica:"))
    algorithms.eaSimple(pop_2, toolbox, 0.5, 0.2, 100, stats_2, halloffame=hof_2, verbose=True, reportFrec=10)
    
    print("")
    print("HallOfFame:")
    print (hof_2[0])
    print("")
    #plotting(hof_2[0])

    print("")
    print("Rule 1: IF ( " + str(hof_1[0]) + " ) (Class = Iris-setosa)")
    print("Rule 2: ELSE IF ( " + str(hof_2[0]) + " ) (Class = Iris-versicolor)")
    print("Rule 3: ELSE (Class = Iris-virginica)")
    return pop_1, stats_1, hof_1, pop_2, stats_2, hof_2,

if __name__ == "__main__":
    main()

Iris-setosa vs. Iris-versicolor, Iris-virginica:
gen	nevals	avg   	std    	min	max
0  	500   	73.796	33.8613	0  	150
10 	296   	124.702	41.0835	0  	150
20 	284   	135.586	31.2933	0  	150
30 	294   	137.756	29.176 	0  	150
40 	309   	138.328	29.1194	0  	150
50 	303   	139.532	27.323 	0  	150
60 	267   	138.5  	29.4424	0  	150
70 	308   	139.234	29.0267	2  	150
80 	304   	139.644	28.061 	12 	150
90 	299   	138.534	28.1026	26 	150
100	286   	139.624	28.1756	9  	150

HallOfFame:
lt(PetalWidth, 0.6642805312990222)

Iris-versicolor vs. Iris-virginica:
gen	nevals	avg 	std    	min	max
0  	500   	50.9	9.24673	7  	94 
10 	287   	78.776	24.7135	6  	97 
20 	290   	88.748	16.7843	21 	97 
30 	279   	88.39 	17.6547	8  	97 
40 	322   	88.814	16.7759	6  	97 
50 	257   	88.676	17.2681	8  	97 
60 	313   	88.908	16.6966	8  	97 
70 	283   	89.414	15.9044	45 	97 
80 	261   	89.824	15.2759	27 	97 
90 	293   	89.73 	15.0928	27 	97 
100	301   	89.626	15.8327	16 	97 

HallOfFame:
And(Out(PetalWidth, 1.690180751